# UC-Berkeley Milling Dataset
> This notebook demonstrates how we can use PyPHM with the UC-Berkeley Milling Dataset. We also conduct some basic data exploration to help the reader understand the data.

The UC-Berkeley Milling Dataset is a collection of milling machine signals. In short, the dataset is made of 16 cases of milling tools performing cuts in metal. Six cutting parameters were used in the creation of the data: 

* the metal type (either cast iron or steel, labelled as 1 or 2 in the data set, respectively)
* the depth of cut (either 0.75 mm or 1.5 mm)
* the feed rate (either 0.25 mm/rev or 0.5 mm/rev)

Each of the 16 cases is a combination of the cutting parameters (for example, case one has a depth of cut of 1.5 mm, a feed rate of 0.5 mm/rev, and is performed on cast iron). 

The cases are made up of individual cuts from when the tool is new to degraded or worn. There are 167 cuts (called 'runs' in the documentation) amongst all 16 cases. Many of the cuts are accompanied by a measure of flank wear (VB). We'll use this to label the cuts as either as healthy, degraded, or worn.

Finally, six signals were collected during each cut: acoustic emission (AE) signals from the spindle and table; vibration from the spindle and table; and AC/DC current from the spindle motor. The signals were collected at 250 Hz and each cut has 9000 sampling points, for a total signal length of 36 seconds.

The dataset is available from the [NASA Prognostic Data Repository](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#milling). Consider citing the creators of the dataset if you use it in academic research.

# Quickstart
Use PyPHM to quickly download and prepare the dataset.

**Only if** you are running in Google Colab, install PyPHM.

In [ ]:
# ONLY RUN IF YOU'RE USING GOOGLE COLAB
!pip install pyphm


Import PyPHM, numpy, pandas, and pathlib.

In [ ]:
from pyphm.datasets.milling import MillingPrepMethodA
import numpy as np
import pandas as pd
from pathlib import Path

We will use `MillingPrepMethodA` to prepare the dataset. This method is similar to that described in Cheng et al. [cheng2019multisensory] and von Hahn et al. [].

Instantiate the `MillingPrepMethodA` class. Dowload the dataset by setting `download` to `True`. Keep the sliding window length (`window_size`) to 64, and the amount each window moves (`stride`) to 64. These are also the default values, if you leave these variables blank.

In [ ]:
# define the location of where the raw data folders will be kept.
# e.g. the milling data will be in path_data_raw_folder/milling/ 
path_data_raw_folder = Path(Path.cwd().parent / 'data/raw/' )
print(path_data_raw_folder)

# create the path_data_raw_folder if it does not exist
path_data_raw_folder.mkdir(parents=True, exist_ok=True)

# instantiate the MillingPrepMethodA class and download data if it does not exist
mill = MillingPrepMethodA(root=path_data_raw_folder, download=True)

## Create Numpy Arrays
With the dataprep class instantiated, we can now prepare the dataset by calling methods from the class.

First, let's call the `create_xy_arrays` method. This will create two numpy arrays. The `x` array contains the windowed signals from the milling machine. The `y` array contains the labels and meta-data for each window.

In [ ]:
x, y = mill.create_xy_arrays()
print("x.shape", x.shape)
print("y.shape", y.shape)

As can be seen, the `x` array has a shape of `(11570, 64, 6)`, representing `[no. samples, window length, no. signals]`. 

There are six signals in each sample (you can read more about the types of signals in the Data Exploration section, below). From index 0, to index 5, the signals are:

In [ ]:
mill.signal_names

If you want to access the spindle vibration (`vib_spindle`) signal, you can index the numpy array like this:

In [ ]:
# index into the 'vib_spindle' signal and show the shape
x[:,:,2].shape

The `y` array has a similar shape to `x`, but instead of signals, it contains the label and meta-data for each window. The shape of `y` is like this: `[no. samples, window length, no. label columns]`.

There are three label columns in `y_labels_ids_times`:
* Label column 1 (index 0): The label, or tool class, for each sample (window). Either `0` for healthy, `1` for degraded, or `2` for worn (failed). More details provided in the Data Exploration section.
* Label column 2 (index 1): The ID of the sample. e.g. `1_0` for the first sample of the first window.
* Label column 3 (index 2): The timesteps for each data point in each sample.

To access the label for each sample, you can index the `y` array like this:

In [ ]:
# use the colon in the first index to get all of the 11570 samples
# use 0 in the second index to only get the first label for each data point of each sample
# use 0 in the third index to get the label (either 0, 1, or 2)
y_label = y[:,0,0]
print("y_label.shape", y_label.shape)
y_label

The data in the `y` array is stored with a dtype of `<u32`, a unicode string. You likely want to convert the label column to an integer, like this:

In [ ]:
y_label.astype(int)

In this example, each window (sample) has a length of 64. The signals are collected at 250 Hz. This means that the timesteps are spaced by 1/250 = 0.004 seconds. Here is what this looks like:

In [ ]:
# show the timestep column, converted to `float` first
y[0,:,2].astype(float)

## Create Pandas Dataframe
The method, `create_xy_dataframe`, creates a flat pandas dataframe from the above `x` and `y` arrays. This flat dataframe is amenable to feature engineering using the [tsfresh](https://github.com/blue-yonder/tsfresh) feature engineering library.

Let's call the `create_xy_dataframe` method.

In [ ]:
df = mill.create_xy_dataframe()
df.head()

You can look at the shape of the dataframe and see that it is quite long. The dataframe is each sample stacked ontop of each other.

In [ ]:
df.shape

Now we can slice and dice the dataframe, using pandas, to get what you want.

Let's select the cut number 56 from the dataframe.

In [ ]:
df[df['cut_no'] == 56].head()

In the Data Exploration section we'll discuss what the `cut_no`, `case`, and the various signal columns are.

# Data Exploration
Let's explore the UC-Berkeley Milling Dataset in detail. First, we should understand what milling is.

## What is milling?

In milling, a rotary cutter removes material as it moves along a work piece. Most often, milling is performed on metal – it's metal machining – and that’s what is happening in the UC Berkeley milling dataset.    

The picture below demonstrates a face milling procedure. The cutter is progressed forward while rotating. As the cutter rotates, the tool inserts “bite” into the metal and remove it. (image modified from [Wikipedia](https://commons.wikimedia.org/wiki/File:Fraisage_surfacage.svg#/media/File:Fraisage_surfacage.svg))

![face milling](./images/face_milling.png)

Over time, the tool inserts wear. Specifically, the flank of the tool wears, as shown below (image from author). In the UC Berkeley milling data set the flank wear (VB) is measured as the tool wears.

![flank wear](./images/flank_wear.png)

## Dataset Structure
Of course, the PyPHM module absracts aways the complexity of loading and preprocessing the dataset. However, it is good for us to understand the structure of the dataset. To do that, we'll load the dataset manually.

The UC Berkeley milling dataset is contained in a structured MATLAB array. We can load the .mat file using the scipy.io module and the loadmat function.

Since we have already downloaded the dataset, we can load it from the local directory.

In [ ]:
# import matplotlib, seaborn, and scipy
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio # for reading matlab files

# load the data from the matlab file
mill_matfile = sio.loadmat(path_data_raw_folder / 'milling' / 'mill.mat',struct_as_record=True)

The data is stored as a dictionary. Let's look to see what it is made of.

In [ ]:
# show some of the info from the matlab file
print('Keys in the matlab dict file: \n', mill_matfile.keys(), '\n')

Only the 'mill' part of the dictionary contains useful information. We'll put that in a new numpy array called 'data.

In [ ]:
# check to see what m['mill'] is
print(type(mill_matfile['mill']))

# store the 'mill' data in a seperate np array
data = mill_matfile['mill']

This raw "data" array can also be accessed through the `MillingPrepMethodA` by calling the `load_mat` method. Let's do that and use it going forward.

In [ ]:
data = mill.load_mat()

We now want to see what the 'data' array is made up of.

In [ ]:
# store the field names in the data np array in a tuple, l
l = data.dtype.names
print('List of the field names:\n',l)

Here's more information on each of the fields in the 'data' array:

* `case` - The case number (from 1 to 16).
* `run` - The run number (from 1 to 167).
* `VB` - The measured flank wear, in mm.
* `time` - Cumulative duration of after each run, in minutes (restarts after each case).
* `DOC` - The depth of cut, in mm (either 0.75 mm or 1.5 mm)
* `feed` - The feed rate, in mm/rev (either 0.25 mm/rev or 0.5 mm/rev)
* `material` - The material used (either cast iron or steel, labelled as 1 or 2 in the data set, respectively)
* `smAC` - AC spindle motor current
* `smDC` - DC spindle motor current
* `vib_table` - Table vibration
* `vib_spindle` - Spindle vibration
* `AE_table` - Acoustic emission from the table
* `AE_spindle` - Acoustic emission from the spindle

In [ ]:
min(data['run'][0].shape)